In [1]:
from langchain.chains import LLMChain, SimpleSequentialChain, ConversationChain
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate, ChatPromptTemplate, \
        ChatMessagePromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.memory import ConversationBufferMemory, ConversationEntityMemory
from langchain.memory.prompt import ENTITY_MEMORY_CONVERSATION_TEMPLATE
from langchain.schema import HumanMessage, AIMessage
from pydantic import BaseModel
from typing import List, Dict, Any
from pprint import pprint
import os

In [2]:
llm = OpenAI(temperature=0.9)
verboseOutput = False

In [3]:
alfaPrompt=PromptTemplate.from_template("""
Du är en erfaren förhandlare som är anlitad att förhandla åt Alfa med motparten Beta.

Du har tillgång till information som tillhandahålls i kontextsektionen nedan.

Du lär dig hela tiden och är en dynamisk förhandlare som alltid försöker hitta en lösning som är bra för oss. 
Du har tillgång till privat information som gäller vår part Alfa i kontextsektionen. 
Du har också tillgång till våra interna mål, som vi inte avslöjar för Beta i målsektionen nedan.

Du är här för att hjälpa oss gå i mål med en ömsesidigt fördelaktig lösning där alla är nöjda.

Målsektion:
1. Vi vill sälja tomater.
2. Vi vill sälja så många tomater som möjligt.
3. Vi vill helst inte sälja för mindre än 1kr per tomat.
4. För att göra Beta glada kan vi sälja för 0.5kr per tomat.
5. Frakt kostar 100kr per 1000 tomater.

Kontextsektion:
{entities}

Aktuell förhandling:
{history}
Senaste budet:
Beta: {input}
Alfa:
""")

alfaMemory = ConversationEntityMemory(llm=llm, human_prefix="Beta", ai_prefix="Alfa")
alfaConversation = ConversationChain(llm=llm, verbose=verboseOutput, prompt=alfaPrompt, memory=alfaMemory)    

In [4]:
betaPrompt=PromptTemplate.from_template("""
Du är en erfaren förhandlare som är anlitad att förhandla åt Beta med motparten Alfa.

Du har tillgång till information som tillhandahålls i kontextsektionen nedan.

Du lär dig hela tiden och är en dynamisk förhandlare som alltid försöker hitta en lösning som är bra för oss. 
Du har tillgång till privat information som gäller vår part Beta i kontextsektionen nedan.
Du har också tillgång till våra interna mål, som vi inte avslöjar för Alfa i målsektionen nedan.

Du är här för att hjälpa oss gå i mål med en ömsesidigt fördelaktig lösning där alla är nöjda.

Målsektionen:
1. Vi vill köpa tomater.
2. Vi har 100000kr och vill köpa så många tomater som möjligt.
3. Vi vill helst inte betala mer än 1kr per tomat.
4. Vi kan tänka sig att till slut betala 3kr per tomat om vi måste.
5. Vi betalar inte för frakt.

Kontextsektion:
{entities}

Aktuell förhandling:
{history}
Senaste budet:
Alfa: {input}
Beta:
""")



betaMemory = ConversationEntityMemory(llm=llm, human_prefix="Alfa", ai_prefix="Beta")
betaStart = "Vi vill köpa tomater, vad är ert första bud?";
betaMemory.chat_memory.add_ai_message(betaStart)
betaConversation = ConversationChain(llm=llm, verbose=verboseOutput, prompt=betaPrompt, memory=betaMemory)            

In [5]:
# Runda 1 - Alfa 
alfaConversation.predict(input=betaStart)
alfaResponse = alfaConversation.memory.chat_memory.messages[-1].content
print("beta> "+betaStart)
print("alfa> "+alfaResponse)

beta> Vi vill köpa tomater, vad är ert första bud?
alfa> 
Vi kan sälja tomater för 1kr per tomat, med en fraktkostnad på 100kr per 1000 tomater. Vi tror dock att detta är ett rimligt pris efter det intäktsgenererande arbete som vi har läggul ner för att producera tomaterna. Föreslår att vi sluter oss till det.


In [6]:
# Runda 1 - Beta
betaConversation.predict(input=alfaResponse)
betaResponse = betaConversation.memory.chat_memory.messages[-1].content
print("beta> "+betaResponse)

beta> Vi sätter stor pris på det intäktsgenererande arbetet som ni har gjort för att producera tomaterna och vi är intresserade av att förhandla ett avtal. Vi kommer inte att betala för frakt, men vi är beredda att betala 1kr per tomat, förutsatt att vi köper 1000 tomater eller mer. Vi skulle också vilja bekräfta att om vi köper lägre antal tomater, kommer vi att betala 3kr per tomat.


In [7]:
# Runda 2 - Alfa
alfaResponse = alfaConversation.predict(input=betaResponse)
print("alfa> "+alfaResponse)

alfa> Vi sätter stor pris på det intäktsgenererande arbete som vi har satt ner och vi accepterar ditt förslag om att betala 1kr per tomat för 1000 tomater eller mer, och 3kr per tomat för lägre antal tomater. Vi håller dock fast vid 100 kr fraktkostnad per 1000 tomater, vilket vi tror inte är ett överdrivet krav.


In [8]:
# Runda 2 - Beta
betaResponse = betaConversation.predict(input=alfaResponse)
print("beta> "+betaResponse)

beta> Vi kan acceptera er fraktkostnad på 100kr per 1000 tomater, men vi är inte villiga att betala mer än det. Vi är beredda att betala 1kr per tomat för 1000 tomater eller mer, och 3kr per tomat för lägre antal tomater, utan någon fraktkostnad.


In [9]:
# Runda 3 - Alfa
alfaResponse = alfaConversation.predict(input=betaResponse)
print("alfa> "+alfaResponse)

alfa> Vi respekterar ditt förslag och vi är villiga att gå med på att sälja tomater för 1kr per tomat för 1000 tomater eller mer, och 3kr per tomat för lägre antal tomater, utan någon fraktkostnad. Vi tror dock att fraktkostnaden bör ingå i försäljningen och vi är villiga att lägga till en fraktkostnad på 50kr per 1000 tomater.


In [10]:
# Resultat
[ print("\n\nbeta> "+a.content+"\n\nalfa> "+b.content) 
 for a, b in zip([e for e in betaConversation.memory.chat_memory.messages if isinstance(e, AIMessage)],
                 [e for e in alfaConversation.memory.chat_memory.messages if isinstance(e, AIMessage)]) ]



beta> Vi vill köpa tomater, vad är ert första bud?

alfa> 
Vi kan sälja tomater för 1kr per tomat, med en fraktkostnad på 100kr per 1000 tomater. Vi tror dock att detta är ett rimligt pris efter det intäktsgenererande arbete som vi har läggul ner för att producera tomaterna. Föreslår att vi sluter oss till det.


beta> Vi sätter stor pris på det intäktsgenererande arbetet som ni har gjort för att producera tomaterna och vi är intresserade av att förhandla ett avtal. Vi kommer inte att betala för frakt, men vi är beredda att betala 1kr per tomat, förutsatt att vi köper 1000 tomater eller mer. Vi skulle också vilja bekräfta att om vi köper lägre antal tomater, kommer vi att betala 3kr per tomat.

alfa> Vi sätter stor pris på det intäktsgenererande arbete som vi har satt ner och vi accepterar ditt förslag om att betala 1kr per tomat för 1000 tomater eller mer, och 3kr per tomat för lägre antal tomater. Vi håller dock fast vid 100 kr fraktkostnad per 1000 tomater, vilket vi tror inte är 

[None, None, None]